In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
with open("payload.txt", "r+") as f:
    data = f.read()

lista_payload = data.split('\n')[1:-1]

In [4]:
MAX_ANGLE = np.pi
MIN_ANGLE = -np.pi

MAX_VOLTAGE = 30000
MIN_VOLTAGE = 0

MAX_CURRENT = 1000
MIN_CURRENT = -10

MAX_MOTOR_CURRENT = 1000
MIN_MOTOR_CURRENT = 0

MAX_BATTERY = 100
MIN_BATTERY = 0

In [5]:
columns=["Posicao 0", "Posicao 1", "Posicao 2", "Posicao 3", "Posicao 4", "Posicao 5", "Posicao 6", "Posicao 7",
                           "OFF", "MANUAL", "AUTO", "COMISSIONAMENTO",
                           "Dia", "Backtracking", 
                           "Bateria OK", "Bateria Insuficiente", "Bateria Minimo", "Bateria Critico",
                           "OK", "Botao de Emergencia", "Horario não Configurado", "Falha na Comunicação", "Posição fora do Intervalo",
                           "Velocidade Baixa", "Undervoltage no Motor", "Eixo Bloqueado", "Sobrecorrente no Motor", "Sobrecorrente na Entrada",
                           "Undervoltage na Entrada", "Alarme 11", "Alarme 12", "Alarme 13", "Alarme 14", "Alarme 15",
                           "TensaoPainel", "CorrentePainel", "PosicaoAngular", "AnguloCalculado", "CorrenteMotor", "EstadoBateria", "Reserva"]

df = pd.DataFrame(columns=columns)

for payload in lista_payload:
    payload_convertido = []

    ### Posicao de Seguranca
    valor = int(payload[0:3], 2)
    aux_list = 8*[False]
    aux_list[valor] = True
    payload_convertido += aux_list

    ### Estado
    valor = int(payload[3:5], 2)
    aux_list = 4*[False]
    aux_list[valor] = True
    payload_convertido += aux_list

    ### Momento do Dia
    payload_convertido += [bool(payload[5])]

    ### Backtracking
    payload_convertido += [bool(payload[6])]

    ### Alarmes Bateria
    valor = int(payload[7:9], 2)
    aux_list = 4*[False]
    aux_list[valor] = True
    payload_convertido += aux_list

    ### Alarmes
    valor = int(payload[9:13], 2)
    aux_list = 16*[False]
    aux_list[valor] = True
    payload_convertido += aux_list

    ### Tensão do Painel
    valor = int(payload[13:29], 2)/(2**16-1)*(MAX_VOLTAGE - MIN_VOLTAGE) + MIN_VOLTAGE
    payload_convertido += [valor]

    ### Corrente do Painel
    valor = int(payload[29:45], 2)/(2**16-1)*(MAX_CURRENT - MIN_CURRENT) + MIN_CURRENT
    payload_convertido += [valor]

    ### Posicao Angular
    valor = int(payload[45:61], 2)/(2**16-1)*(MAX_ANGLE - MIN_ANGLE) + MIN_ANGLE
    payload_convertido += [valor]

    ### Angulo Calculado
    valor = int(payload[61:77], 2)/(2**16-1)*(MAX_ANGLE - MIN_ANGLE) + MIN_ANGLE
    payload_convertido += [valor]

    ### Corrente do Motor
    valor = int(payload[77:93], 2)/(2**16-1)*(MAX_MOTOR_CURRENT - MIN_MOTOR_CURRENT) + MIN_MOTOR_CURRENT
    payload_convertido += [valor]

    ### Estado da Bateria
    valor = int(payload[93:101], 2)/(2**8-1)*(MAX_BATTERY - MIN_BATTERY) + MIN_BATTERY
    payload_convertido += [valor]

    ### Reserva
    valor = int(payload[101:], 2)
    payload_convertido += [valor]

    df = df.append(pd.DataFrame([payload_convertido], columns=columns), ignore_index=True)


In [6]:
df.to_csv("payload_descompactado.csv")

In [7]:
df

,Posicao 0,Posicao 1,Posicao 2,Posicao 3,Posicao 4,Posicao 5,Posicao 6,Posicao 7,OFF,MANUAL,...,Alarme 13,Alarme 14,Alarme 15,TensaoPainel,CorrentePainel,PosicaoAngular,AnguloCalculado,CorrenteMotor,EstadoBateria,Reserva
0,True,False,False,False,False,False,False,False,True,False,...,False,False,False,0.000000,-0.013275,0.092280,0.087199,9.994659,0.000000,0
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,3.662165,-0.013275,0.092759,0.087199,9.994659,0.392157,0
2,False,False,False,True,False,False,False,False,True,False,...,False,False,False,3.662165,-0.013275,0.092951,0.087199,9.994659,0.784314,0
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,3.662165,-0.013275,0.092472,0.087199,9.994659,1.176471,0
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,0.000000,-0.013275,0.091417,0.087199,9.994659,1.568627,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,False,False,False,False,False,False,True,False,True,False,...,False,False,False,23916.685740,21.994507,-0.229765,-0.219506,674.998093,0.784314,0
1403,True,False,False,False,False,False,False,False,False,False,...,False,False,False,23891.966125,21.994507,-0.201769,-0.200235,674.998093,1.176471,0
1404,False,False,False,True,False,False,False,False,False,True,...,False,False,False,23891.966125,22.996262,-0.202153,-0.199468,674.998093,1.568627,0
1405,False,False,False,True,False,False,False,False,False,False,...,False,False,False,23942.778668,20.992752,-0.174062,-0.180198,674.998093,1.960784,0
